# Sentimental Analysis of news headline for stock price prediction

In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from textblob import TextBlob
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [29]:
df = pd.read_csv('../input/stockmarket2/BSESN (1).csv')
df.head(5)


,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-07-01,4263.109863,4301.770020,4247.660156,4300.859863,4300.859863,0.0
1,1997-07-02,4302.959961,4395.310059,4295.399902,4333.899902,4333.899902,0.0
2,1997-07-03,4335.790039,4393.290039,4299.970215,4323.459961,4323.459961,0.0
3,1997-07-04,4332.700195,4347.589844,4300.580078,4323.819824,4323.819824,0.0
4,1997-07-07,4326.810059,4391.009766,4289.490234,4291.450195,4291.450195,0.0


In [30]:
df1 = pd.read_csv('../input/stockmarket1/india-news-headlines.csv')
df1.head()

,publish_date,headline_category,headline_text
0,20010101,sports.wwe,win over cena satisfying but defeating underta...
1,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
2,20010102,unknown,Fissures in Hurriyat over Pak visit
3,20010102,unknown,America's unwanted heading for India?
4,20010102,unknown,For bigwigs; it is destination Goa


In [31]:
df1 = df1.rename(columns = {"publish_date":"Date"})
del df1['headline_category']
df1.head(5)

,Date,headline_text
0,20010101,win over cena satisfying but defeating underta...
1,20010102,Status quo will not be disturbed at Ayodhya; s...
2,20010102,Fissures in Hurriyat over Pak visit
3,20010102,America's unwanted heading for India?
4,20010102,For bigwigs; it is destination Goa


In [32]:
df1['Date'].dtype

dtype('int64')

In [33]:
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%Y%m%d")
df["Date"]=df["Date"].astype(int)
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,19970701,4263.109863,4301.770020,4247.660156,4300.859863,4300.859863,0.0
1,19970702,4302.959961,4395.310059,4295.399902,4333.899902,4333.899902,0.0
2,19970703,4335.790039,4393.290039,4299.970215,4323.459961,4323.459961,0.0
3,19970704,4332.700195,4347.589844,4300.580078,4323.819824,4323.819824,0.0
4,19970707,4326.810059,4391.009766,4289.490234,4291.450195,4291.450195,0.0
...,...,...,...,...,...,...,...
5855,20201009,40226.250000,40585.359375,40066.539063,40509.488281,40509.488281,16600.0
5856,20201012,40716.949219,40905.488281,40387.398438,40593.800781,40593.800781,17600.0
5857,20201013,40592.539063,40786.820313,40461.968750,40625.511719,40625.511719,11000.0
5858,20201014,40623.191406,40880.250000,40279.550781,40794.738281,40794.738281,15000.0


In [34]:
df['Date'].dtype

dtype('int64')

In [35]:
df2 = df1.groupby('Date')['headline_text'].agg(list)
df2.shape

(7080,)

In [36]:
df2.head()

Date
20010101    [win over cena satisfying but defeating undert...
20010102    [Status quo will not be disturbed at Ayodhya; ...
20010103    [Powerless north India gropes in the dark, Thi...
20010104    [The string that pulled Stephen Hawking to Ind...
20010105    [Light combat craft takes India into club clas...
Name: headline_text, dtype: object

In [37]:
mer = pd.merge(df,df2,how='inner',on='Date')
mer

,Date,Open,High,Low,Close,Adj Close,Volume,headline_text
0,20010101,NaN,NaN,NaN,NaN,NaN,NaN,[win over cena satisfying but defeating undert...
1,20010102,3953.219971,4028.570068,3929.370117,4018.879883,4018.879883,0.0,[Status quo will not be disturbed at Ayodhya; ...
2,20010103,3977.580078,4067.659912,3977.580078,4060.020020,4060.020020,0.0,"[Powerless north India gropes in the dark, Thi..."
3,20010104,4180.970215,4180.970215,4109.549805,4115.370117,4115.370117,0.0,[The string that pulled Stephen Hawking to Ind...
4,20010105,4116.339844,4195.009766,4115.350098,4183.729980,4183.729980,0.0,[Light combat craft takes India into club clas...
...,...,...,...,...,...,...,...,...
4836,20200624,35679.738281,35706.550781,34794.929688,34868.980469,34868.980469,26600.0,[I never thought I had a voice until today: Vi...
4837,20200625,34525.390625,35081.609375,34499.781250,34842.101563,34842.101563,24600.0,[Truck firms look for new export markets to se...
4838,20200626,35144.781250,35254.878906,34910.339844,35171.269531,35171.269531,24800.0,[Containment zone residents slam high prices c...
4839,20200629,34926.949219,35032.359375,34662.058594,34961.519531,34961.519531,18300.0,[6 hot and stylish bikini looks of Katrina Kai...


In [38]:
headline = []
headline.append(mer.iloc[1,7])
headline

[['Status quo will not be disturbed at Ayodhya; says Vajpayee',
  'Fissures in Hurriyat over Pak visit',
  "America's unwanted heading for India?",
  'For bigwigs; it is destination Goa',
  'Extra buses to clear tourist traffic',
  'Dilute the power of transfers; says Riberio',
  'Focus shifts to teaching of Hindi',
  'IT will become compulsory in schools',
  "Move to stop freedom fighters' pension flayed",
  'Gilani claims he applied for passport 2 years ago',
  "Parivar dismisses PM's warning",
  'India; Pak exchange lists of N-plants',
  "Will Qureshi's return really help the govt?",
  "PM's tacit message: Put Ram tample on hold",
  "Text of the Prime Minister's article",
  'NCW to focus on violence against women',
  "BBC's reality TV will focus on AIIMS",
  'Jaitley firm on legal reforms',
  'Hoshangabad farmers have enough water',
  'BJP jumps into rail track conversion issue',
  "America's unwanted heading for India?",
  'State may take AP to court over Krishna',
  'This official

In [39]:
mer['headline_text'] = mer['headline_text'].map(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', str(x)))

In [40]:
mer['headline_text']

0        win over cena satisfying but defeating undert...
1        Status quo will not be disturbed at Ayodhya s...
2        Powerless north India gropes in the dark Thin...
3        The string that pulled Stephen Hawking to Ind...
4        Light combat craft takes India into club clas...
                              ...                        
4836     I never thought I had a voice until today Vid...
4837     Truck firms look for new export markets to se...
4838     Containment zone residents slam high prices c...
4839     6 hot and stylish bikini looks of Katrina Kai...
4840     Detective Byomkesh Bakshy Edge of Tomorrow Fi...
Name: headline_text, Length: 4841, dtype: object

In [41]:
mer.shape

(4841, 8)

In [42]:
mer.dropna(inplace = True)

In [43]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [44]:
mer['Subjectivity'] = mer['headline_text'].apply(getSubjectivity)
mer['Polarity'] = mer['headline_text'].apply(getPolarity)

In [45]:
mer.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,headline_text,Subjectivity,Polarity
1,20010102,3953.219971,4028.570068,3929.370117,4018.879883,4018.879883,0.0,Status quo will not be disturbed at Ayodhya s...,0.286859,0.143590
2,20010103,3977.580078,4067.659912,3977.580078,4060.020020,4060.020020,0.0,Powerless north India gropes in the dark Thin...,0.392857,0.089286
3,20010104,4180.970215,4180.970215,4109.549805,4115.370117,4115.370117,0.0,The string that pulled Stephen Hawking to Ind...,0.445360,0.093039
4,20010105,4116.339844,4195.009766,4115.350098,4183.729980,4183.729980,0.0,Light combat craft takes India into club clas...,0.480553,0.264024
5,20010108,4164.759766,4206.720215,4101.529785,4120.430176,4120.430176,0.0,Sangh Parivar Babri panel up the ante Frontru...,0.400343,-0.011266


In [46]:
mer['Label'] = mer['Polarity'].apply(lambda x : 1 if x >= 0.1 else 0)

In [47]:
mer.head(30)

,Date,Open,High,Low,Close,Adj Close,Volume,headline_text,Subjectivity,Polarity,Label
1,20010102,3953.219971,4028.570068,3929.370117,4018.879883,4018.879883,0.0,Status quo will not be disturbed at Ayodhya s...,0.286859,0.143590,1
2,20010103,3977.580078,4067.659912,3977.580078,4060.020020,4060.020020,0.0,Powerless north India gropes in the dark Thin...,0.392857,0.089286,0
3,20010104,4180.970215,4180.970215,4109.549805,4115.370117,4115.370117,0.0,The string that pulled Stephen Hawking to Ind...,0.445360,0.093039,0
4,20010105,4116.339844,4195.009766,4115.350098,4183.729980,4183.729980,0.0,Light combat craft takes India into club clas...,0.480553,0.264024,1
5,20010108,4164.759766,4206.720215,4101.529785,4120.430176,4120.430176,0.0,Sangh Parivar Babri panel up the ante Frontru...,0.400343,-0.011266,0
6,20010109,4114.740234,4166.839844,4101.009766,4125.310059,4125.310059,0.0,Financier Bharat Shah held for alleged links ...,0.501934,-0.003837,0
7,20010110,4151.580078,4155.859863,4035.439941,4047.639893,4047.639893,0.0,Fend for yourselves Pande tells doctors Burea...,0.000000,0.000000,0
8,20010111,4066.129883,4077.540039,4006.540039,4027.129883,4027.129883,0.0,shreyas gopal scores ton as karnataka stay on...,0.300000,0.250000,1
9,20010123,4277.830078,4321.750000,4272.870117,4296.689941,4296.689941,0.0,Sonia takes on the Hindutva brigade Congress ...,0.291841,0.062937,0
10,20010124,4320.509766,4339.410156,4295.709961,4326.419922,4326.419922,0.0,Govt extends J K ceasefire by a month Peace h...,0.260526,0.091228,0


In [48]:
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,19970701,4263.109863,4301.770020,4247.660156,4300.859863,4300.859863,0.0
1,19970702,4302.959961,4395.310059,4295.399902,4333.899902,4333.899902,0.0
2,19970703,4335.790039,4393.290039,4299.970215,4323.459961,4323.459961,0.0
3,19970704,4332.700195,4347.589844,4300.580078,4323.819824,4323.819824,0.0
4,19970707,4326.810059,4391.009766,4289.490234,4291.450195,4291.450195,0.0


In [49]:
keep_columns = [ 'Date','Close','Adj Close', 'Subjectivity', 'Polarity','Label']
data = mer[keep_columns]
data

,Date,Close,Adj Close,Subjectivity,Polarity,Label
1,20010102,4018.879883,4018.879883,0.286859,0.143590,1
2,20010103,4060.020020,4060.020020,0.392857,0.089286,0
3,20010104,4115.370117,4115.370117,0.445360,0.093039,0
4,20010105,4183.729980,4183.729980,0.480553,0.264024,1
5,20010108,4120.430176,4120.430176,0.400343,-0.011266,0
...,...,...,...,...,...,...
4836,20200624,34868.980469,34868.980469,0.385245,0.087935,0
4837,20200625,34842.101563,34842.101563,0.395552,0.028492,0
4838,20200626,35171.269531,35171.269531,0.377957,0.060128,0
4839,20200629,34961.519531,34961.519531,0.395725,0.065173,0


In [50]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d') 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
data.head(5)

,Date,Close,Adj Close,Subjectivity,Polarity,Label
1,2001-01-02,4018.879883,4018.879883,0.286859,0.143590,1
2,2001-01-03,4060.020020,4060.020020,0.392857,0.089286,0
3,2001-01-04,4115.370117,4115.370117,0.445360,0.093039,0
4,2001-01-05,4183.729980,4183.729980,0.480553,0.264024,1
5,2001-01-08,4120.430176,4120.430176,0.400343,-0.011266,0


In [52]:
data.set_index('Date', inplace=True)

In [54]:
#Create the feature data set
X = data
X = np.array(X.drop(['Label'],1))
#Create the target data set
y = np.array(data['Label'])

In [55]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [56]:
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [57]:
predictions = model.predict(x_test)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [58]:
print( classification_report(y_test, predictions) )

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       832
           1       1.00      0.64      0.78       122

    accuracy                           0.95       954
   macro avg       0.97      0.82      0.88       954
weighted avg       0.96      0.95      0.95       954

